# AWS Privilege Escalation Using IAM Login Profile

Detect when a user creates or updates a login profile for another user and then a successful AWS Console login is performed from the same IP address within a short time window. This can indicate an attempt to escalate privileges by unauthorized users.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource:("iam.amazonaws.com" "signin.amazonaws.com")
(
(
eventSource:"iam.amazonaws.com"
eventName:("CreateLoginProfile" "UpdateLoginProfile")
)
or
(
eventSource:"signin.amazonaws.com"
eventName:"ConsoleLogin"
responseElements.ConsoleLogin:"Success"
)
)
not errorCode:*
| groupbycount(
recipientAccountId,
sourceIPAddress
)
| where @q.count > 1

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName